In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v2 import CNN_v2

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v2((480, 640, 4), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v2/weights/"

In [7]:
model_names = ["cnn_v2_1e_6_100",
          "cnn_v2_1e_6_110",
          "cnn_v2_1e_6_50",
          "cnn_v2_1e_6_70",
          "cnn_v2_5e_5_60",
          "cnn_v2_5e_5_70",
          "cnn_v2_5e_5_80",
          "cnn_v2_1e_6_100",
          "cnn_v2_1e_6_110",
          "cnn_v2_1e_6_120",
          "cnn_v2_1e_6_135", 
          "cnn_v2_1e_6_next_01_0.75",
          "cnn_v2_1e_6_next_03_0.76",
          "cnn_v2_1e_6_next_05_0.77",
          "cnn_v2_1e_6_next_06_0.78",
          "cnn_v2_1e_6_next_07_0.82",
          "cnn_v2_1e_6_next_11_0.83",
          "cnn_v2_1e_6_next_12_0.84",
          "cnn_v2_1e_6_next_13_0.84",
          "cnn_v2_1e_6_next_16_0.84",
          "cnn_v2_1e_6_next_17_0.85",
          "cnn_v2_1e_6_next_1_01_0.78",
          "cnn_v2_1e_6_next_1_02_0.84",
          "cnn_v2_1e_6_next_1_06_0.86",
          "cnn_v2_1e_6_next_22_0.86",
          "cnn_v2_1e_6_next_28_0.86",
          "cnn_v2_1e_6_next_30_0.87"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [CNN_v2((480, 640, 4), split_into_parts=20) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7066 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7053 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7080 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7070 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7127 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 4s 4s/step - loss: 0.7067 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 5s 5s/step - loss: 0.7098 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 5s 5s/step - loss: 0.7101 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 5s 5s/step - loss: 0.7086 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 5s 5s/step - loss: 0.7068 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 6s 6s/step - loss: 0.7104 - acc: 1.0000
Epoch 1/1
1/1 [======

In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model CNN/CNN_v2/weights/cnn_v2_1e_6_100 behaved:
For cut-of-point 0.5 had accuracy 85.38454901873286:
[  4  -9  -6  -8  -5   0   2  -4   6  -2  -2   0  -9  -3   1  -5  -3  -1
  -2   0  -4  -3  -5  -6  -5  -7  -7  -5   2  -2  -4 -11  -6  -1   5   0
   6  -2   1  -1  11  -4   4   4   2  -2   0   2   2   3  -5  -7   4 -11
  -7   5   4   8   0 -10  -3  -5  -4  -2  -5  -2  -2  -9 -10 -10  -3  -3
  -2  -6  -6  -3  -7 -11  -7  -7  -5  -5  -5  -5  -7  -3   0  -2   7   2
   0  -4   1 -10  -5 -10  -2  -8  -2 -10   8  -2   6  -2   2   0  -7   6
 -10   0  -2  -4 -12 -10  -8  -4  -6  -5  -3  -4  -9   0  -3  -2  -3   1
   1  -2  -2   0  -4   5  -3   0 -12  -1  -1  -6  -7 -10  -5  -4  -9   5
   5  -2  -4   6  -2  -3  -2  -8   0  -3  -8  -4  -6   2   0  -8   1  -6
  -5  -6   0  -6  -2 -10  -6  -2   1   3  -4   0   0  -4  -2  -2   4  -3
  -6 -10  -7  -7  -9  -4 -10   6  -1  -3  -3   4  -2  -5  -7   3   3   1
   4  -5  -5  -8  -4   1  -2   1   3  -5  -6 -11  -1  -1 -10  -7 -13   6
  -4   2  -1   3   4 